In [1]:
import pandas as pd 
import numpy as np
import cv2
import torch
import comet_ml
import os
import glob
import re
import tensorflow as tf
from functools import partial
from retinaface import RetinaFace
import torch
from natsort import natsorted
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms

import torchmetrics
import pytorch_lightning as pl
from tqdm import tqdm
import matplotlib.pyplot as plt

2022-12-07 14:56:46.436160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 14:56:47.224623: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 14:56:49.013585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/silver22/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.7/lib64
2022-12-07 14:56:49.013819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic lib

### Инфо по версии CUDA

In [2]:
def get_cuda_info():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    print("CUDA: ",torch.version.cuda)
    print("IS CUDA AVAILABLE: ",torch.cuda.is_available())
    
get_cuda_info()


cuda
CUDA:  11.7
IS CUDA AVAILABLE:  True


### Формируем датасет самых и менее доминантных персон

In [5]:
def get_name_of_folders_with_video():
    set_of_video = ['IS1000a','IS1001a','IS1001b','IS1001c','IS1003b','IS1003d','IS1006b','IS1008a',
                'IS1008b','IS1008c','IS1008d']
    return set_of_video

def get_csv_data(type_data=None):
    annot_data = pd.read_csv('dome_annotations_M1.csv')
    dome_dataset = pd.read_csv('dome_dataset_M1.csv')
    annot_data = annot_data.join(dome_dataset['FMD'])
    annot_data = annot_data.join(dome_dataset['FLD'])
    if type_data == "FMD":
        annot_data = annot_data[annot_data['FMD'] == 1]
    elif type_data == "FLD":
        annot_data = annot_data[annot_data['FLD'] == 1]
    
    return annot_data
        
        
print(get_csv_data("FMD"))

       name  start (sec)  end (sec)  A11  A12  A13  A14  A21  A22  A23  A24  \
0   IS1000a            0        300    4    1    3    2    4    1    3    2   
1   IS1000a          300        600    3    1    4    2    3    1    4    2   
2   IS1000a          600        900    1    4    2    3    1    2    4    3   
4   IS1000a         1200       1500    4    1    3    2    3    1    4    2   
6   IS1001a          300        600    1    2    4    3    1    2    4    3   
7   IS1001a          600        900    1    2    3    4    1    2    3    4   
9   IS1001b          300        600    1    3    4    2    1    2    4    3   
12  IS1001b         1200       1500    1    2    3    4    1    3    2    4   
13  IS1001b         1500       1800    1    3    2    4    1    2    3    4   
14  IS1001b         1800       2100    1    2    3    4    1    2    3    4   
18  IS1001c          900       1200    1    2    3    4    1    2    3    4   
19  IS1001c         1200       1445    1    3    2  

### Получаем кадры из видео 

In [ ]:
def create_images_from_video(image_output_path, video_input_path, frame_skip = 5, 
                             duration = 60, type_data = "FMD"):
    set_of_video = get_name_of_folders_with_video()
    csv_data = get_csv_data(type_data)
    
    for name_video_folder in set_of_video:
        part_data = csv_data.loc[csv_data['name'] == name_video_folder]
        start_time = part_data['start (sec)'].values * 25
        end_time = part_data['end (sec)'].values * 25
        video_input_path = f"{video_input_path}/{name_video_folder}"
        len_of_data = len(part_data)
        
        for number_person in range(0,4):
            vidcap = cv2.VideoCapture(f'amicorpus/{name_video_folder}/video/{name_video_folder}.Closeup{number_person+1}.avi')
            success,image = vidcap.read()
            count = 0
            index = 0
            step_unit_time = 0
            duration_frames = duration * 25
            bin_count = 0
            while success:
                temp_data = part_data.iloc[index].values[3:7].tolist()
                dom_person = 0
                if type_data == "FMD":
                    dom_person = temp_data.index(1)
                elif type_data == "FLD":
                    dom_person = temp_data.index(4)
                if count % frame_skip == 0 and count >= start_time[index] and count <= end_time[index]:
                    if count % duration_frames == 0:
                        step_unit_time = int(count / duration_frames)
                        dir_path = f"{image_output_path}/{name_video_folder}/{step_unit_time}unit_time_domperson{dom_person}"
                        if not os.path.exists(dir_path):
                            os.makedirs(dir_path)
                    dir_path = f"{image_output_path}/{name_video_folder}/{step_unit_time}unit_time_domperson{dom_person}/person{number_person}"
                    if not os.path.exists(dir_path):
                        bin_count = 0
                        os.makedirs(dir_path) 
                    bin_count_txt = f'{bin_count:0012b}'
                    cv2.imwrite(f"{dir_path}/{bin_count_txt}.jpg", image)     # save frame as JPEG file
                    bin_count += 1
                success,image = vidcap.read()
                if count >= end_time[index] - 1 and len_of_data-1 > index:
                    index += 1
                count += 1
                        
create_images_from_video(image_output_path="test_output",video_input_path="amicorpus")

In [ ]:
# for name_video_folder in set_of_video:
#   part_data = annot_data.loc[annot_data['name'] == name_video_folder]
#   len_of_data = len(part_data)
#   start_time = part_data['start (sec)'].values * 25 #number of frames
#   end_time = part_data['end (sec)'].values * 25
#   for number_person in range(0,4):
#     vidcap = cv2.VideoCapture(f'amicorpus/{name_video_folder}/video/{name_video_folder}.Closeup{number_person+1}.avi')
#     success,image = vidcap.read()
#     count = 0
#     index = 0
#     bin_count = 0
#     time_frames = 0
#     while success:
#       temp_data = part_data.iloc[index].values[3:7].tolist()
#       dom_persone = temp_data.index(1)
#       if count % 5 == 0 and count >= start_time[index] and count <= end_time[index] :
#         #print(count)
#         if count % 1500 == 0:
#           time_frames = int(count / 1500)
#           dir1 = os.path.join("NewImages/", f"{name_video_folder}", f"{time_frames}min_domperson{dom_persone}")
#           if not os.path.exists(dir1):
#             os.mkdir(dir1)
#         dir2 = os.path.join(f"NewImages/{name_video_folder}", f"{time_frames}min_domperson{dom_persone}", f"person{number_person}")
#         if not os.path.exists(dir2):
#           bin_count = 0
#           os.mkdir(dir2)
#         bin_count_txt = f'{bin_count:0012b}'
#         cv2.imwrite(f"NewImages/{name_video_folder}/{time_frames}min_domperson{dom_persone}/person{number_person}/{bin_count_txt}.jpg", image)
#         bin_count += 1
#       success,image = vidcap.read()
#       #print(count," ",dom_persone)
#       if count >= end_time[index] - 1 and len_of_data-1 > index:
#         index += 1
#       count += 1

In [78]:
def get_pixel(img, center, x, y):
	new_value = 0
	try:
		if img[x][y] >= center:
			new_value = 1
	except:
		pass

	return new_value

# Function for calculating LBP
def lbp_calculated_pixel(img, x, y):
	center = img[x][y]
	val_ar = []

	val_ar.append(get_pixel(img, center, x-1, y-1))
	
	val_ar.append(get_pixel(img, center, x-1, y))
	
	val_ar.append(get_pixel(img, center, x-1, y + 1))

	val_ar.append(get_pixel(img, center, x, y + 1))
	
	val_ar.append(get_pixel(img, center, x + 1, y + 1))
	
	val_ar.append(get_pixel(img, center, x + 1, y))
	
	val_ar.append(get_pixel(img, center, x + 1, y-1))
	
	val_ar.append(get_pixel(img, center, x, y-1))
	
	power_val = [1, 2, 4, 8, 16, 32, 64, 128]

	val = 0
	
	for i in range(len(val_ar)):
		val += val_ar[i] * power_val[i]
		
	return val

def LBP(img, align = False, shape = (128, 128)):
    height, width = shape
    img_lbp = np.zeros((height, width),
				np.uint8)
    for i in range(0, height):
        for j in range(0, width):
            img_lbp[i, j] = lbp_calculated_pixel(img, i, j)
    return img_lbp

def get_features_LBP(img, align = False, shape = (128, 128)):
	if img.sum() == 0:
		return np.zeros(256)
	else:
		feature = LBP(img, align, shape)
		mean_img = np.mean(feature, axis = 0)
		std_img = np.std(feature, axis = 0)
		feature = np.concatenate((mean_img, std_img))
		return feature
	#print(feature)
	#new_feature = np.array([mean_img, std_img])
	#print(mean_img)
# def get_features():
#     #s = start_time()
#     X_data = []
#     y_data = []
#     for name_video_folder in set_of_video:
#         for idx, dom_name in enumerate(['NonDominant','MostDominant']):
#             for number_person in range(1,5):
#                 count = 1
#                 temp_arr = np.zeros([128,1],dtype=int)
#                 list_of_files = glob.glob(f'Face_Dominant_32x32/{dom_name}/{name_video_folder}/person{number_person}/*')
#                 for image_path in tqdm(list_of_files):
#                     gray = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
#                     if count % 300 == 0:
#                         X_data.append(temp_arr)
#                         y_data.append(idx)
#                         temp_arr = np.zeros([128,1],dtype=int)
#                     else:
#                         gray_img = LBP(gray)
#                         print(np.shape(gray_img))
#                         mean_img = gray_img.mean(0)
#                         std_img = gray_img.std(0)
#                         print(np.shape(mean_img[0]))
                        
#                         temp_arr = np.append(temp_arr,LBP(gray),1)
#                     count += 1
#                     # print(np.shape(X_data))
                    

#     # for image_path in list_of_files:
#     #     if count % 1000 == 999:
#     #         print(file_path," - ", count)
#     #     gray = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
#     #     arr_img = LBP(gray)
#     #     X_data.append(arr_img)
#     #     y_data.append(file_path)
#     #     count += 1

#     return X_data, y_data

# X,y = get_features()

In [ ]:
def face_detector(image_input_path, image_output_path, resize=False, shape=(128,128), face_extractor=None):
    face_extractor = RetinaFace 
    # if face_extractor == "RetinaFace":
    #     face_extractor = RetinaFace
    # elif face_extractor == "MTCNN":
    #     face_extractor = MTCNN
    # elif face_extractor == "FaceNet":
    #     face_extractor = FaceNet
    # elif face_extractor == "LBP":
    #     face_extractor = partial(LBP, shape=shape)
        
    
    set_of_video = get_name_of_folders_with_video()
    for name_video_folder in set_of_video:
        list_of_dirs = os.listdir(f"{image_input_path}/{name_video_folder}")
        for dir_name in list_of_dirs:
            for number_person in range(0,4):
                dir_path = f"{image_output_path}/{name_video_folder}/{dir_name}/person{number_person}"
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
                list_of_path_images = glob.glob(f"{image_input_path}/{name_video_folder}/{dir_name}/person{number_person}/*")
                
                for path_image in tqdm(list_of_path_images):
                    image_name = path_image.split("/")[-1]
                    faces = face_extractor.extract_faces(img_path = path_image, align = False)
                    if faces:
                        for idx, face in enumerate(faces):
                            if resize:
                                face = cv2.resize(face, shape)
                            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                            cv2.imwrite(f"{dir_path}/{idx}id_{image_name}", face)
                    else:
                        black_image = np.zeros(shape, dtype=np.uint8)
                        cv2.imwrite(f"{dir_path}/0id_{image_name}", black_image)
                
        

face_detector(image_input_path="NewImages", image_output_path="test_output_2", resize=True, face_extractor=None)  

In [ ]:
# for name_video_folder in set_of_video:
#     list_of_files = glob.glob(f'NewImages/{name_video_folder}/*')
#     for files in list_of_files:
#         dir_1 = f'Face_Dominant128x128/{files[10:]}'
#         if not os.path.exists(dir_1):
#             os.mkdir(dir_1)
#         for number_person in range(0,4):
#             dir_2 = f'Face_Dominant128x128/{files[10:]}/person{number_person}'
#             if not os.path.exists(dir_2):
#                 os.mkdir(dir_2)
            
#             list_of_images = glob.glob(f'{files}/person{number_person}/*')
#             for image_file in tqdm(list_of_images):
#                 faces = RetinaFace.extract_faces(img_path = image_file, align = False)
#                 if faces:
#                     img_res = cv2.resize(faces[0],(128,128))
#                     cv2.imwrite(f'Face_Dominant128x128/{image_file[10:]}',cv2.cvtColor(img_res, cv2.COLOR_BGR2GRAY))
#                 else:
#                     black_image = np.zeros((128,128,1), np.uint8)
#                     cv2.imwrite(f'Face_Dominant128x128/{image_file[10:]}',black_image)

    

In [ ]:
def feature_extraction(image_input_path, feature_extractor=None, save_csv=False, csv_path=None):
    feature_extractor = get_features_LBP
    dict_template = {"person0_features":[],"person1_features":[],"person2_features":[],"person3_features":[],"id_video":[],"labels":[]}
    csv_result = pd.DataFrame(dict_template)
    
    set_of_video = get_name_of_folders_with_video()
    for name_video_folder in set_of_video:
        list_of_dirs = os.listdir(f"{image_input_path}/{name_video_folder}")
        for dir_name in tqdm(list_of_dirs):
            dom_person = dir_name[-1]
            dict_csv = {"person0_features":[],"person1_features":[],"person2_features":[],"person3_features":[],"id_video":name_video_folder,"labels":dom_person}
            for number_person in range(0,4):
                person_features = []
                list_of_path_images = glob.glob(f"{image_input_path}/{name_video_folder}/{dir_name}/person{number_person}/*")
                for path_image in list_of_path_images:
                    image = cv2.imread(path_image, cv2.IMREAD_GRAYSCALE)
                    feature = feature_extractor(image)
                    person_features.append(feature)
                mean_person_features = np.mean(person_features, axis=0)
                dict_csv[f"person{number_person}_features"].append(mean_person_features)
            csv_temp = pd.DataFrame(dict_csv)
            csv_result = pd.concat([csv_result, csv_temp], ignore_index=True)
    if save_csv:
        csv_result.to_csv(csv_path)
        
    
        

feature_extraction(image_input_path="Face_Dominant128x128", save_csv=True, csv_path="csv_LBP.csv")

In [2]:
def get_csv(csv_path):
    return pd.read_csv(csv_path)

In [7]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = csv_file
        self.person0 = self.data["person0_features"].apply(lambda x: np.array(x[1:-1].split(), dtype=np.float32))
        self.person1 = self.data["person1_features"].apply(lambda x: np.array(x[1:-1].split(), dtype=np.float32))
        self.person2 = self.data["person2_features"].apply(lambda x: np.array(x[1:-1].split(), dtype=np.float32))
        self.person3 = self.data["person3_features"].apply(lambda x: np.array(x[1:-1].split(), dtype=np.float32))
        self.person0_norm = self.person0.apply(lambda x: x/np.linalg.norm(x)).apply(lambda x: np.nan_to_num(x))
        self.person1_norm = self.person1.apply(lambda x: x/np.linalg.norm(x)).apply(lambda x: np.nan_to_num(x))
        self.person2_norm = self.person2.apply(lambda x: x/np.linalg.norm(x)).apply(lambda x: np.nan_to_num(x))
        self.person3_norm = self.person3.apply(lambda x: x/np.linalg.norm(x)).apply(lambda x: np.nan_to_num(x))
        self.transform = transform
    
    def __getitem__(self, idx):
        data = self.data.iloc[idx]
        person0 = self.person0[idx]
        person1 = self.person1[idx]
        person2 = self.person2[idx]
        person3 = self.person3[idx]
        label = int(data['labels'])
    
        return person0, person1, person2, person3, label
        
    
    def __len__(self):
        return len(self.data)
    

# csv_res1 = pd.read_csv("csv_LBP.csv")
# csv_res1['person0_features'].apply(lambda x: np.array(x[1:-1].split(), dtype=np.float32))



In [8]:
def train_val_test_split(csv_file):
    train_csv = csv_file[(csv_file['id_video'] != 'IS1003b') & (csv_file['id_video'] != 'IS1001c') & (csv_file['id_video'] != 'IS1008c')]
    train_csv = train_csv.reset_index(drop=True)
    val_csv = csv_file[(csv_file['id_video'] == 'IS1003b') | (csv_file['id_video'] == 'IS1001c')]
    val_csv = val_csv.reset_index(drop=True)
    test_csv = csv_file[(csv_file['id_video'] == 'IS1008c')]
    test_csv = test_csv.reset_index(drop=True)
    return train_csv, val_csv, test_csv

In [9]:
result_csv = get_csv("csv_LBP.csv")

train_csv, val_csv, test_csv = train_val_test_split(result_csv)

print(len(train_csv), len(val_csv), len(test_csv))

# train_csv = result_csv[(result_csv['id_video'] != 'IS1003b') & (result_csv['id_video'] != 'IS1001c') & (result_csv['id_video'] != 'IS1008c')]
# train_csv = train_csv.reset_index(drop=True)
# val_csv = result_csv[(result_csv['id_video'] == 'IS1003b') | (result_csv['id_video'] == 'IS1001c')]
# val_csv = val_csv.reset_index(drop=True)
# test_csv = result_csv[(result_csv['id_video'] == 'IS1008c')]
# test_csv = test_csv.reset_index(drop=True)

train_loader = DataLoader(FeaturesDataset(train_csv),batch_size=1, num_workers=8)
val_loader = DataLoader(FeaturesDataset(val_csv),batch_size=1, num_workers=8)
test_loader = DataLoader(FeaturesDataset(test_csv),batch_size=1, num_workers=8)

131 15 15


In [ ]:
next(iter(train_loader))

In [3]:
name_of_features = ['AU01_r','AU02_r','AU04_r','AU05_r','AU06_r','AU07_r','AU09_r','AU10_r',
                    'AU12_r','AU14_r','AU15_r','AU17_r','AU20_r','AU23_r','AU25_r','AU26_r',
                    'AU45_r','AU01_c','AU02_c','AU04_c','AU05_c','AU06_c','AU07_c',
                    'AU09_c','AU10_c','AU12_c','AU14_c','AU15_c','AU17_c','AU20_c','AU23_c',
                    'AU25_c','AU26_c','AU28_c','AU45_c']

In [5]:
set_of_video = ['IS1000a','IS1001a','IS1001b','IS1001c','IS1003b','IS1003d','IS1006b','IS1008a',
                'IS1008b','IS1008c','IS1008d']

In [15]:
test = pd.read_csv('NewOpenFaceFeatures2/IS1000a/7min_domperson1/person1/person1.csv')
#test[name_of_features] = 0
test.loc[test['confidence'] < 0.8, name_of_features] = 0
test

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.0,0.88,1,0.217457,0.033246,-0.975504,-0.182038,0.028498,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2,0,0.0,0.88,1,0.212280,0.039451,-0.976412,-0.191070,0.044660,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,3,0,0.0,0.88,1,0.217850,0.070070,-0.973464,-0.195546,0.086792,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,4,0,0.0,0.88,1,0.232961,0.072879,-0.969751,-0.184878,0.092050,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,0.0,0.88,1,0.241321,0.095292,-0.965756,-0.176681,0.114356,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,0,0.0,0.88,1,0.206368,0.247157,-0.946745,-0.061561,0.209678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
296,297,0,0.0,0.88,1,0.209831,0.298733,-0.930983,-0.021205,0.258392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
297,298,0,0.0,0.77,1,0.206486,0.305500,-0.929534,-0.012050,0.261493,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,299,0,0.0,0.98,1,0.187627,0.245146,-0.951157,-0.038141,0.196185,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
column_names = ["person0_features", "person1_features", "person2_features",
                "person3_features", "id_video", "labels"]

result_csv = pd.DataFrame(columns = column_names)
for name_video_folder in set_of_video:
    #time_folder = glob.glob(f'NewOpenFaceFeatures2/{name_video_folder}/*')
    time_folder = natsorted(os.listdir(f'NewOpenFaceFeatures2/{name_video_folder}/'))
    #print(natsorted(time_folder))
    for time_f_path in time_folder:
        person_arr = []
        id_video = ""
        label = ""
        temp2_csv = pd.DataFrame(columns = column_names)
        for number_person in range(0,4):
            temp_csv = pd.read_csv(f'NewOpenFaceFeatures2/{name_video_folder}/{time_f_path}/person{number_person}/person{number_person}.csv')
            temp_csv.loc[temp_csv['confidence'] < 0.8, name_of_features] = 0
            mean = np.mean(temp_csv[name_of_features].values.tolist(),axis=0)
            std = np.std(temp_csv[name_of_features].values.tolist(),axis=0)
            person_arr.append(np.concatenate([mean, std]))
        temp2_csv["person0_features"] = [person_arr[0]]
        temp2_csv["person1_features"] = [person_arr[1]]
        temp2_csv["person2_features"] = [person_arr[2]]
        temp2_csv["person3_features"] = [person_arr[3]]
        temp2_csv["id_video"] = name_video_folder
        temp2_csv["labels"] = person_num = re.findall(r'[0-9]$',time_f_path)[0]
        result_csv = pd.concat([result_csv,temp2_csv])
        
        #print(result_csv["person0_features"].values[0])
            #person_arr.append(temp_csv[name_of_features].values.tolist())
result_csv = result_csv.reset_index(drop=True)


NameError: name 'set_of_video' is not defined

In [6]:
result_csv.to_csv("OpenFaceFeatures.csv",index=False)

In [36]:
result_csv = pd.read_csv("OpenFaceFeatures.csv")

In [6]:
print(type(result_csv["person0_features"].values[0]))

<class 'numpy.ndarray'>


In [7]:
#Create Dataset for pytorch
class FeaturesDataset(torch.utils.data.Dataset):
    """Some Information about MyDataset"""
    def __init__(self,csv_var):
        super(FeaturesDataset, self).__init__()
        self.csv_var = csv_var

    def __getitem__(self, index):
        person0 = self.csv_var['person0_features'][index].astype(np.float32)
        person1 = self.csv_var['person1_features'][index].astype(np.float32)
        person2 = self.csv_var['person2_features'][index].astype(np.float32)
        person3 = self.csv_var['person3_features'][index].astype(np.float32)
        label = int(self.csv_var["labels"][index])
        return person0, person1, person2, person3, label

    def __len__(self):
        return len(self.csv_var)

In [8]:
# X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(train_set.fname.values, train_set.label_encoded.values, 
                                                #   test_size=0.2, random_state=42)

train_csv = result_csv[(result_csv['id_video'] != 'IS1003b') & (result_csv['id_video'] != 'IS1001c') & (result_csv['id_video'] != 'IS1008c')]
train_csv = train_csv.reset_index(drop=True)
val_csv = result_csv[(result_csv['id_video'] == 'IS1003b') | (result_csv['id_video'] == 'IS1001c')]
val_csv = val_csv.reset_index(drop=True)
test_csv = result_csv[(result_csv['id_video'] == 'IS1008c')]
test_csv = test_csv.reset_index(drop=True)

train_loader = DataLoader(FeaturesDataset(train_csv),batch_size=1, num_workers=8)
val_loader = DataLoader(FeaturesDataset(val_csv),batch_size=1, num_workers=8)
test_loader = DataLoader(FeaturesDataset(test_csv),batch_size=1, num_workers=8)

In [9]:
print(len(val_loader))

15


In [10]:
next(iter(train_loader))

[tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 tensor([[0.3926, 0.2126, 0.0577, 0.1203, 0.2548, 0.6150, 0.0751, 0.1703, 0.1835,
          0.4268, 0.2937, 0.4099, 0.1523, 0.2212, 0.4920, 0.5147, 0.2141, 0.1667,
          0.2333, 0.3133, 0.1200, 0.2267, 0.0067, 0.1467, 0.3400, 0.3100, 0.4933,
          0.2667, 0.1867, 0.1133, 0.2467, 0.2700, 0.2533, 0.0700, 0.2100, 0.7018,
          0.5484, 0.1180, 0.3207, 0.3774, 0.6417, 0.1775, 0.2770, 0.3416, 0.5497,
          0.5317, 0.4553, 0.3313, 0.5260, 0.6184, 0.7112, 0.4080, 0.3727, 0.4230,
          0.4638, 0.3250, 0.4187, 0.0814, 0.3538, 0.4737, 0.4625, 0.5000, 0.4422,
          0.3896, 0.3170, 0.4311, 0.4440, 0.4349, 0.2551, 0.4073]]),
 tensor([[0.1852, 0.1529, 0.1733, 0.0611, 0.

In [11]:
import torch.nn.functional as F
class Model(pl.LightningModule):
    def __init__(self, in_features = 70, out_features = 4):
        super(Model, self).__init__()
        # self.l1   =  nn.Linear(in_features,in_features)
        # self.l2   =  nn.Linear(in_features,in_features*2)
        # self.l3   =  nn.Linear(in_features*2*4,in_features*2*4)
        # self.l4   =  nn.Linear(in_features*2*4,out_features)
        self.l1   =  nn.Linear(70,70)
        self.l2   =  nn.Linear(70,128)
        self.l3   =  nn.Linear(512,512)
        self.l4   =  nn.Linear(512,4)
        
#         self.l1   =  nn.Linear(256,256)
#         self.l2   =  nn.Linear(256,128)
#         self.l3   =  nn.Linear(512,512)
#         self.l4   =  nn.Linear(512,4)
        self.dropout = nn.Dropout(0.2)
        
        #metrics
        self.f1_train = torchmetrics.F1Score(num_classes=4)
        self.f1_val = torchmetrics.F1Score(num_classes=4)
        self.acc_train = torchmetrics.Accuracy(num_classes=4)
        self.acc_val = torchmetrics.Accuracy(num_classes=4)
        self.pr_train = torchmetrics.Precision(num_classes=4)
        self.pr_val = torchmetrics.Precision(num_classes=4)
        self.recall_train = torchmetrics.Recall(num_classes=4)
        self.recall_val = torchmetrics.Recall(num_classes=4)
        self.f1_test = torchmetrics.F1Score(num_classes=4)
        self.acc_test = torchmetrics.Accuracy(num_classes=4)
        self.pr_test = torchmetrics.Precision(num_classes=4)
        self.recall_test = torchmetrics.Recall(num_classes=4)
    
        #loss
        self.loss = torch.nn.CrossEntropyLoss()
        
    def forward(self, person1, person2, person3, person4):
        p1 = self.dropout(self.l2(self.dropout(self.l1(person1))))
        p2 = self.dropout(self.l2(self.dropout(self.l1(person2))))
        p3 = self.dropout(self.l2(self.dropout(self.l1(person3))))
        p4 = self.dropout(self.l2(self.dropout(self.l1(person4))))
        com = torch.cat((p1,p2,p3,p4),dim=1)
        com = torch.flatten(com)
        x = self.dropout(self.l3(com))
        x = self.dropout(self.l4(x))
        return x #F.softmax(x) #Убрать     
    
    def training_step(self, batch, batch_idx):

        p1,p2,p3,p4, y = batch
        y_pred = self(p1,p2,p3,p4)
        
        loss = self.loss(y_pred,y[0])
        
        y_pred = y_pred.view(1,-1)
  
        
        self.f1_train(y_pred, y)
        self.acc_train(y_pred, y)
        self.recall_train(y_pred, y)
        self.pr_train(y_pred, y)
        self.log("train_loss", loss,prog_bar=True,on_epoch=True)
        # self.log("train_acc", self.acc_train, on_step=True, on_epoch=False, prog_bar=True)
        # self.log("train_f1", self.f1_train, on_step=True, on_epoch=False, prog_bar=True)
        # self.f1_train(y_pred, y)
        # self.acc_train(y_pred, y)

        return loss

    def training_epoch_end(self, outs):
        f1 = self.f1_train.compute()
        acc = self.acc_train.compute()
        pr = self.pr_train.compute()
        recall_var = self.recall_train.compute()
        self.log('train_f1_epoch', f1)
        self.log('train_acc_epoch', acc)
        self.log('train_precision_epoch', pr)
        self.log('train_recall_epoch', recall_var)
        #self.log("train_acc", self.train_acc.compute())
    
    def validation_step(self, batch, batch_idx):
        p1,p2,p3,p4, y = batch
        y_pred = self(p1,p2,p3,p4)
        #print(y)
        #print("pred: ",y_pred,"target: ",y)
        
        loss = self.loss(y_pred, y[0])
        
        y_pred = y_pred.view(1,-1)
  
        self.f1_val(y_pred, y)
        self.acc_val(y_pred, y)
        self.pr_val(y_pred, y)
        self.recall_val(y_pred, y)
        self.log("valid_loss", loss, prog_bar=True)
        #self.log("valid_acc", self.acc_val, on_step=True, on_epoch=False, prog_bar=True)
        #self.log("valid_f1", self.f1_val, on_step=True, on_epoch=False, prog_bar=True)
        #print("valid_loss", loss)
        #self.valid_acc.update(y_pred, y)
        #self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        # self.f1_val(y_pred, y)
        # self.acc_val(y_pred, y)
        return loss

    def validation_epoch_end(self, outputs):
        f1 = self.f1_val.compute()
        acc = self.acc_val.compute()
        pr = self.pr_val.compute()
        recall_var = self.recall_val.compute()
        self.log('val_f1_epoch', f1)
        self.log('val_acc_epoch', acc)
        self.log('val_precision_epoch', pr)
        self.log('val_recall_epoch', recall_var)
    
    def test_step(self, batch, batch_idx):
        p1,p2,p3,p4, y = batch
        y_pred = self(p1,p2,p3,p4)
        #print(y)
        #print("pred: ",y_pred,"target: ",y)
        
        loss = self.loss(y_pred, y[0])
        
        y_pred = y_pred.view(1,-1)
  
        self.f1_test(y_pred, y)
        self.acc_test(y_pred, y)
        self.pr_test(y_pred, y)
        self.recall_test(y_pred, y)
        self.log("test_loss", loss, prog_bar=True)
        #self.log("valid_acc", self.acc_val, on_step=True, on_epoch=False, prog_bar=True)
        #self.log("valid_f1", self.f1_val, on_step=True, on_epoch=False, prog_bar=True)
        #print("valid_loss", loss)
        #self.valid_acc.update(y_pred, y)
        #self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        # self.f1_val(y_pred, y)
        # self.acc_val(y_pred, y)
        return loss
    
    def test_epoch_end(self, outputs):
        f1 = self.f1_test.compute()
        acc = self.acc_test.compute()
        pr = self.pr_test.compute()
        recall_var = self.recall_test.compute()
        self.log('test_f1_epoch', f1)
        self.log('test_acc_epoch', acc)
        self.log('test_precision_epoch', pr)
        self.log('test_recall_epoch', recall_var)
    
    # def test_step(self, batch, batch_idx):
    #     p1,p2,p3,p4, y = batch
    #     #print(self(p1,p2,p3,p4))
    #     y_pred = torch.flatten(self(p1,p2,p3,p4))
        
    #     loss = F.cross_entropy(y_pred.float(),y[0])
    #     y_pred = torch.tensor([torch.argmax(y_pred)])
    #     self.test_acc.update(y_pred, y) 
    #     self.log("test_loss", loss, prog_bar=True) 
    #     self.log("test_acc", self.test_acc.compute(), prog_bar=True) 
    #     return loss


    # def test_epoch_end(self, outputs):
    #     avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
    #     logs = {"test_loss": avg_loss}
    #     return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}
        
    # def predict_step(self, batch, batch_idx, dataloader_idx=0):
    #     x = batch
    #     y_pred = self(x)
    #     y_pred = torch.argmax(y_pred, dim=1)
    #     y_pred = self.le.inverse_transform(y_pred.cpu().detach().numpy())
    #     return y_pred

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        return optimizer 

In [12]:
from pytorch_lightning.loggers import CometLogger

model = Model(in_features=70)
comet_logger = CometLogger(
    api_key="hU25TlG9e0dRKvtdx1SwbIrHK",
    save_dir="logs",
    project_name="dominant",
    experiment_name="dominant8_LBPFeatures",
    # Optional
)
# experiment = comet_ml.Experiment(
#     api_key="hU25TlG9e0dRKvtdx1SwbIrHK",
#     project_name="dominant",
# )
trainer = pl.Trainer(max_epochs=10,accelerator="gpu",devices=1,logger=comet_logger,gradient_clip_val=1)

trainer.fit(model, train_dataloaders=train_loader,val_dataloaders=val_loader)

#comet_logger.existing_experiment.log_asset("dominant8_LBPFeatures.ckpt")

trainer.test(dataloaders=test_loader)
#trainer.test(val_data)

CometLogger will be initialized in online mode
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name         | Type             | Params
---------------------------------------------------
0  | l1           | Linear           | 5.0 K 
1  | l2           | Linear           | 9.1 K 
2  | l3           | Linear           | 262 K 
3  | l4           | Linear           | 2.1 K 
4  | dropout      | Dropout          | 0     
5  | f1_train     | F1Score          | 0     
6  | f1_val       | F1Score          | 0     
7  | acc_train    | Accuracy         | 0     
8  | acc_val      | Accuracy         | 0     
9  | pr_train     | Precision        | 0     
10 | pr_val       | Precision        | 0     
11 | recall_train | Recall           | 0     
12 | recall_val   | Recall           | 0     
13 | f1_test      | F1Score          | 0     
14 | acc_test   

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/philippshemetov/dominant/59451b0a6f2d4c759898a56aeac003ea



Validation: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=10` reached.
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/philippshemetov/dominant/59451b0a6f2d4c759898a56aeac003ea
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_acc_epoch [10]       : (0.5572519302368164, 0.652671754360199)
COMET INFO:     train_f1_epoch [10]        : (0.5572519302368164, 0.652671754360199)
COMET INFO:     train_loss_epoch [10]      : (1.1819325685501099, 3.29459285736084)
COMET INFO:     train_loss_step [26]       : (-0.0, 6.413330554962158)
COMET INFO:     train_precision_epoch [10] : (0.5572519302368164, 0.652671754360199)
COMET INFO:     train_recall_epoch [10]    : (0.5572519302368164, 0.652671754360199)
COMET INFO:     val_acc_epoch [10]         : (0.0, 0.44736841320991516)
COMET INFO:     val_f1_epoch [10]        

Testing: 0it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/philippshemetov/dominant/59451b0a6f2d4c759898a56aeac003ea

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/philippshemetov/dominant/59451b0a6f2d4c759898a56aeac003ea
COMET INFO:   Metrics:
COMET INFO:     test_acc_epoch       : 0.20000000298023224
COMET INFO:     test_f1_epoch        : 0.20000001788139343
COME

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.20000000298023224
      test_f1_epoch         0.20000001788139343
        test_loss           10.845248222351074
  test_precision_epoch      0.20000000298023224
    test_recall_epoch       0.20000000298023224
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 10.845248222351074,
  'test_f1_epoch': 0.20000001788139343,
  'test_acc_epoch': 0.20000000298023224,
  'test_precision_epoch': 0.20000000298023224,
  'test_recall_epoch': 0.20000000298023224}]

In [13]:
trainer.test(dataloaders=test_loader)

Restoring states from the checkpoint path at logs/dominant/59451b0a6f2d4c759898a56aeac003ea/checkpoints/epoch=9-step=1310.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at logs/dominant/59451b0a6f2d4c759898a56aeac003ea/checkpoints/epoch=9-step=1310.ckpt


Testing: 0it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/philippshemetov/dominant/59451b0a6f2d4c759898a56aeac003ea

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/philippshemetov/dominant/59451b0a6f2d4c759898a56aeac003ea
COMET INFO:   Metrics:
COMET INFO:     test_acc_epoch       : 0.20000000298023224
COMET INFO:     test_f1_epoch        : 0.20000001788139343
COME

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.20000000298023224
      test_f1_epoch         0.20000001788139343
        test_loss           10.845248222351074
  test_precision_epoch      0.20000000298023224
    test_recall_epoch       0.20000000298023224
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 10.845248222351074,
  'test_f1_epoch': 0.20000001788139343,
  'test_acc_epoch': 0.20000000298023224,
  'test_precision_epoch': 0.20000000298023224,
  'test_recall_epoch': 0.20000000298023224}]

In [22]:
%load_ext tensorboard 
%tensorboard --logdir lightning_logs/

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [ ]:
next(iter(train_loader))